In [111]:
model_name="iampanda/zpoint_large_embedding_zh"

In [2]:
import os

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

os.environ['WANDB_DISABLED'] = "true"

In [3]:
# !gdown 1qq5gDuEAqWnoviRBDYfbprLFD-MPQyNo
# !gdown 1mq9CWT9MXsH1Y-ihxVL4uEzufQjpBoer
!gdown 1EI12smTmdi4Mlu_lLNULN6p6g9dYEWql

Downloading...
From: https://drive.google.com/uc?id=1EI12smTmdi4Mlu_lLNULN6p6g9dYEWql
To: /content/processed_data.zip
100% 6.00M/6.00M [00:00<00:00, 47.4MB/s]


In [4]:
!rm -rf processed_data

!unzip -q processed_data.zip

In [5]:
!pip install -q transformers[torch] accelerate -U

!pip install -q datasets

!pip install -q transformers

!pip install -q iterative-stratification

!pip install -q auto-gptq optimum bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 90.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.1/424.1 kB 28.3 MB/s eta 0:00:00
   ━━━━━

In [6]:
import numpy as np

import pandas as pd

import os

from os import path

import glob

from tqdm.notebook import tqdm

from tqdm import tqdm

import string

import copy

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer,IntervalStrategy,BitsAndBytesConfig

from datasets import Dataset,load_dataset, DatasetDict

# from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay



from sklearn.model_selection import train_test_split,KFold

from sklearn.metrics import classification_report,accuracy_score, f1_score

from sklearn.metrics import precision_recall_fscore_support

from sklearn.utils import shuffle

from sklearn.preprocessing import LabelEncoder

from scipy.special import expit

from peft import prepare_model_for_kbit_training

from peft import LoraConfig, get_peft_model,PeftModel


In [7]:
import torch

import torch.nn as nn

import torch.nn.functional as F

In [112]:


models_list_3=[
        model_name,
          ]



models_name_3=[
        model_name,

]

In [113]:
kind="train"

task="track_a"

# langs=["afr","amh","deu","eng","oro","ptbr","rus","som","sum","tir"]
# labels=['Anger','Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise']
# langs=["amh","arq","ary","chn","deu","esp","hau","hin","ibo","kin","mar","orm","pcm","ptbr","ptmz","ron","rus","som","sun","swa","swe","tat","tir","ukr","vmw","yor","ind","jav","xho","zul"]
langs=["chn"]
# langs=["eng"]
# langs=["afr"]


# processed_path=f"processed_data/{kinds[0]}/{tasks[0]}/{langs[3]}.csv"



# train_data=pd.read_csv(processed_path)


In [114]:
all_csv=pd.DataFrame()

for lang in langs:

    processed_path=f"processed_data/{kind}/{task}/{lang}.csv"
    # if kind=="dev":
    #    processed_path=processed_path.replace(lang,lang+task[-2:])
    if not os.path.isfile(processed_path):

      print("not found:",processed_path)

      continue
    train_data=pd.read_csv(processed_path)

    train_data.columns = train_data.columns.str.lower()

    all_csv = pd.concat([all_csv, train_data],ignore_index=True)




In [115]:
train_data=all_csv


In [116]:
train_data=train_data.fillna(0)

In [117]:
float_columns = train_data.select_dtypes(include=['float64']).columns
# Convert those columns to int
train_data[float_columns] = train_data[float_columns].astype('int')

In [118]:
train_data = train_data[

train_data['clean_message'].isnull()==False]

train_data.reset_index()

,index,id,anger,disgust,fear,joy,sadness,surprise,lang,clean_message
0,0,chn_train_track_a_00001,0,0,0,0,0,0,chn,简单说就是有阳性病例查密切接触者时发现了一些很久没有做核酸的人还陆续被查出阳性只能隔离在隔离...
1,1,chn_train_track_a_00002,0,0,0,1,0,0,chn,运镜也好牛 第一次在电视剧中如此密集地感受到 镜头语言 编剧音乐演技都在线
2,2,chn_train_track_a_00003,1,0,0,0,0,0,chn,现在任何事情只要沾上互联网都会变得不幸因为脑残太多总是喜欢毁掉一切美好的事物
3,3,chn_train_track_a_00004,0,0,0,1,0,0,chn,如果能在每个黄昏睡醒后 吃到一堆自己喜欢的东西就好了 看着逐渐被黑暗统治的世界 要下一口炸鸡...
4,4,chn_train_track_a_00005,0,0,0,0,0,0,chn,四五岁时候我天天在家哭唧唧我要去上学为什么不让我去学校那时候农村上学晚也没有太严格要求年龄所...
...,...,...,...,...,...,...,...,...,...,...
2637,2637,chn_train_track_a_02638,1,1,0,0,0,0,chn,反抗的人不对封控也不对反抗出师不好封控不讲人权就这样over
2638,2638,chn_train_track_a_02639,1,1,0,0,1,0,chn,是的报数已经毫无意义反正好多天不去核酸了家族里的年轻人已经好几个感染了这么看七百多的数据就是个笑话
2639,2639,chn_train_track_a_02640,0,0,0,0,1,0,chn,那个时候还在小学当时在午睡摇的有点晕还不知道啥情况就听到老师叫我们快出去现在都大学毕业了太快了
2640,2640,chn_train_track_a_02641,0,0,0,0,0,0,chn,和辛普森案一样欧美法系程序正义和法官的道德瑕疵如果被证明有问题确实是推翻的这和情绪不情绪没有...


In [119]:
# label_columns =['anger', 'fear', 'joy', 'sadness', 'surprise']
label_columns =['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']
# label_columns =['anger', 'disgust', 'fear', 'joy', 'sadness']


In [120]:
train_data['lang'].value_counts()

,count
lang,
chn,2642


In [121]:
sum(train_data['lang'].isna())

0

In [122]:
nan_rows = train_data[train_data.isna().any(axis=1)]
nan_rows

,id,anger,disgust,fear,joy,sadness,surprise,lang,clean_message


In [123]:
train_data['clean_message']=train_data['clean_message'].astype(str)

In [124]:
maxi=train_data['clean_message'].apply(len).max()
maxi

176

In [125]:
train_data[train_data['clean_message'].apply(len) == maxi]

,id,anger,disgust,fear,joy,sadness,surprise,lang,clean_message
2436,chn_train_track_a_02437,1,0,0,1,0,0,chn,评论里其他ip的人少来这里评论谢谢了folded hands folded hands fo...


In [126]:
train_data[train_data['clean_message'].apply(len) == maxi]["clean_message"].values

array(['评论里其他ip的人少来这里评论谢谢了folded hands folded hands folded hands folded hands folded hands folded hands 我没说不放开我反而希望一夜之间全部放开你们赶紧去看演唱会看球赛出国旅游哈反正我家住自建房不怕下水道气溶胶家里囤了够三个人吃两个月的物资朋友是私立医院的老板嘿嘿嘿'],
      dtype=object)

In [127]:
# shuffle data
train_data=train_data.sample(frac=1)
# train_data=train_data.sample(100,random_state=42)


In [128]:
# train valid split
# train,valid=train_test_split(train_data,test_size=0.2,random_state=42)

In [129]:
x_train, y_train = train_data['clean_message'].values.tolist(), train_data[label_columns].values.tolist()

In [130]:
y_train_df = pd.DataFrame(y_train, columns=label_columns)

In [131]:
#update label column name

label_distribution = y_train_df.apply(pd.Series.value_counts).T.fillna(0).astype(int)

label_distribution.columns = ['count_0', 'count_1']
label_distribution['sum'] = label_distribution['count_0'] + label_distribution['count_1']
print(label_distribution)

          count_0  count_1   sum
anger        1464     1178  2642
disgust      2239      403  2642
fear         2571       71  2642
joy          2113      529  2642
sadness      2288      354  2642
surprise     2464      178  2642


In [132]:
device="cuda" if torch.cuda.is_available() else 'cpu'

In [133]:
from sklearn.utils.class_weight import compute_class_weight

counts_0 = label_distribution['count_0'].to_numpy()
counts_1 = label_distribution['count_1'].to_numpy()

# Compute class-specific weights for each label (each class)
class_weights = []
for i in range(len(counts_0)):
    weight = compute_class_weight('balanced', classes=np.array([0, 1]), y=[0] * counts_0[i] + [1] * counts_1[i])
    class_weights.append(weight)

# Convert the list of weights to a tensor
class_weights_tensor = torch.tensor([w[1] for w in class_weights], dtype=torch.float,device=device)
class_weights_tensor

tensor([ 1.1214,  3.2779, 18.6056,  2.4972,  3.7316,  7.4213], device='cuda:0')

In [134]:
MAX_LENGTH = 512

metrics_df = pd.DataFrame(columns=["Models_name", "Precision", "Recall", "F1_score", "Accuracy"])

confusion_matrices = []

In [135]:
quantization_config = BitsAndBytesConfig(

    load_in_4bit=True,

    bnb_4bit_use_double_quant=True,

    bnb_4bit_quant_type="nf4",

    bnb_4bit_compute_dtype=torch.bfloat16,

)

In [136]:
models_list=models_list_3

models_name=models_name_3

x=models_list[0]

In [137]:
token=""

import huggingface_hub

if "gemma" in x or "Phi" in x:

  huggingface_hub.login(token=token)

In [138]:
USE_LORA=False

In [139]:
!pip install -q sentence_transformers

In [140]:
from transformers import AutoModel
from sentence_transformers import SentenceTransformer

In [141]:
del embedding_model
for _ in range(10):
  torch.cuda.empty_cache()

In [142]:


torch.cuda.empty_cache()



print(x)
use_gpu = torch.cuda.is_available()
embedding_model = SentenceTransformer(
        x,
        trust_remote_code=True,
        device="cpu" if not use_gpu else "cuda",
        model_kwargs={
            "torch_dtype": torch.bfloat16 if use_gpu else torch.float32,
            # "attn_implementation": "sdpa"
        },
        # vector_dim must be 12288, 1024, 512, 256
        ## 1024 is recommended
        # set is_text_encoder 'True', if you do not encode image
        # config_kwargs={"is_text_encoder": True, "vector_dim": 1024},
    )
    # We can reduce the max_seq_length from the default of 2048 for faster encoding
embedding_model.max_seq_length = MAX_LENGTH

iampanda/zpoint_large_embedding_zh


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/27.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/7.35M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/7.35M [00:00<?, ?B/s]

In [143]:
def find_target_modules(model):
    # Initialize a Set to Store Unique Layers
    unique_layers = set()

    # Iterate Over All Named Modules in the Model
    for name, module in model.named_modules():
        # Check if the Module Type Contains 'Linear4bit'
        if "Linear4bit" in str(type(module)):
            # Extract the Type of the Layer
            layer_type = name.split('.')[-1]

            # Add the Layer Type to the Set of Unique Layers
            unique_layers.add(layer_type)

    # Return the Set of Unique Layers Converted to a List
    return list(unique_layers)
find_target_modules(embedding_model)

[]

In [144]:
# a=

In [145]:
embedding_model.eval() # embedding_model in training mode (dropout modules are activated)

if USE_LORA:

  # enable gradient check pointing

  embedding_model.gradient_checkpointing_enable()

  target_modules_dict={

      "microsoft/Phi-3.5-mini-instruct":'all-linear',
      "unsloth/Llama-3.2-1B-Instruct":["q_proj","k_proj","v_proj"],
      "unsloth/Qwen2.5-3B-Instruct-bnb-4bit":["q_proj","k_proj","v_proj"],
  }

  target_modules= target_modules_dict.get(x,["q_proj"])



  # enable quantized training

  embedding_model = prepare_model_for_kbit_training(embedding_model)

  config = LoraConfig(

    r=8,

    lora_alpha=32,

    # target_modules=["k_proj"],

    target_modules = target_modules,#TODOOOOOOOOOOOOO

    lora_dropout=0.1,

    bias="none",

    task_type="SEQ_CLS",

  )

  # LoRA trainable version of model

  embedding_model = get_peft_model(embedding_model, config)



  # trainable parameter count

  embedding_model.print_trainable_parameters()

device="cuda" if torch.cuda.is_available() else 'cpu'

In [146]:
device

'cuda'

In [147]:
embedding_model = embedding_model.to(device)

for param in embedding_model.parameters():
    param.data = param.data.contiguous()


In [148]:

# Custom Dataset for batching
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, texts):
        self.texts = texts

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        # Tokenize a single example
        return self.texts[idx]


In [149]:
from torch.utils.data import DataLoader

In [150]:

# Batch extraction function
def extract_embeddings(texts, model, batch_size=32):
    dataset = TextDataset(texts)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    embeddings = []
    model.eval()
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Extracting embeddings"):
            # Move batch to device
            # input_ids = torch.Tensor(batch['input_ids']).squeeze().to(device)
            # attention_mask = torch.Tensor(batch['attention_mask']).squeeze().to(device)

            # Forward pass through the model
            cls_embeddings = model.encode(batch)
            embeddings.append(cls_embeddings)

    return np.vstack(embeddings)

In [151]:
# Extract embeddings for train and validation datasets
x_train_embeddings = extract_embeddings(x_train, embedding_model, batch_size=32)

Extracting embeddings: 100%|██████████| 83/83 [01:27<00:00,  1.06s/it]


In [152]:
np.save("x_train_embeddings.npy",x_train_embeddings)

In [153]:
# Flatten labels for XGBoost (multi-label classification)
y_train_flat = np.array(y_train)

In [154]:
import xgboost as xgb

In [155]:
# xgb_models = {}

# for i, label in enumerate(label_columns):
#     print(f"Training XGBoost for label: {label}")
#     # Use the computed class weights for the positive class
#     scale_pos_weight = class_weights_tensor[i].item()
#     # Initialize and train XGBoost classifier
#     xgb_model = xgb.XGBClassifier(
#         objective='binary:logistic',
#         eval_metric='logloss',
#         use_label_encoder=False,
#         n_estimators=100,
#         learning_rate=0.1,
#         max_depth=6,
#         tree_method='hist',
#         scale_pos_weight=scale_pos_weight,  # Apply the class weight here
#         device=device
#     )
#     xgb_model.fit(x_train_embeddings, y_train_flat[:, i])
#     xgb_models[label] = xgb_model

In [156]:
# Save the XGBoost models if needed
# for label, model in xgb_models.items():
#     model.save_model(f"xgb_model_{label}.json")

In [157]:
# import pickle
# # Save the XGBoost models if needed
# for label, model in xgb_models.items():
#     with open(f"xgb_model_{label}.pkl", 'wb') as f:
#       pickle.dump(model, f)

In [158]:
# !zip xgb_models_pkl.zip xgb_model_*.pkl

In [159]:
!rm -rf svm*

In [160]:
from sklearn.svm import SVC

In [161]:
svm_models = {}

for i, label in enumerate(label_columns):
    print(f"Training SVM for label: {label}")

    # Use the computed class weights for balancing the classes
    class_weight = {0: 1, 1: class_weights_tensor[i].item()}  # Adjust the class weights

    svm_model = SVC(class_weight={0: 1, 1: class_weights_tensor[i].item()})

    svm_model.fit(x_train_embeddings, y_train_flat[:, i])
    svm_models[label] = svm_model

Training SVM for label: anger
Training SVM for label: disgust
Training SVM for label: fear
Training SVM for label: joy
Training SVM for label: sadness
Training SVM for label: surprise


In [162]:
import pickle
# Save the svm models if needed
for label, model in svm_models.items():
    with open(f"svm_model_{label}.pkl", 'wb') as f:
      pickle.dump(model, f)

In [163]:
!zip svm_models_pkl.zip svm_model_*.pkl

  adding: svm_model_anger.pkl (deflated 75%)
  adding: svm_model_disgust.pkl (deflated 75%)
  adding: svm_model_fear.pkl (deflated 75%)
  adding: svm_model_joy.pkl (deflated 75%)
  adding: svm_model_sadness.pkl (deflated 75%)
  adding: svm_model_surprise.pkl (deflated 75%)


In [164]:
kind="dev"

In [165]:
all_csv=pd.DataFrame()

for lang in langs:

    processed_path=f"processed_data/{kind}/{task}/{lang}.csv"
    # if kind=="dev":
    #    processed_path=processed_path.replace(lang,lang+task[-2:])
    if not os.path.isfile(processed_path):

      print("not found:",processed_path)

      continue
    train_data=pd.read_csv(processed_path)

    train_data.columns = train_data.columns.str.lower()

    all_csv = pd.concat([all_csv, train_data],ignore_index=True)
dev_data=all_csv
dev_data=dev_data.fillna(0)
float_columns = dev_data.select_dtypes(include=['float64']).columns

# Convert those columns to int
dev_data[float_columns] = dev_data[float_columns].astype('int')
dev_data = dev_data[dev_data['clean_message'].isnull()==False]
dev_data.reset_index()


,index,id,anger,disgust,fear,joy,sadness,surprise,lang,clean_message
0,0,chn_dev_track_a_00001,0,0,0,0,0,0,chn,害人家看不见张嘴还是pill 王也不知道从哪拿的钱最近手头儿紧我也想挣点
1,1,chn_dev_track_a_00002,0,0,0,0,0,0,chn,秋收冬藏鸟语花香你是来日方长
2,2,chn_dev_track_a_00003,0,0,0,0,0,0,chn,过家家摘野花刺泡野草莓茶籽茶叶花生等收油菜籽稻谷在田里拿禾堆堆稻草屋坐桥上用脚捞上游来的水藻...
3,3,chn_dev_track_a_00004,0,0,0,0,0,0,chn,不管你姓江还姓刘只要是中国法律管的人个人募捐这个事就该摆上台面希望今后再不必看这永无止境子子...
4,4,chn_dev_track_a_00005,0,0,0,0,0,0,chn,我相信你我在海珠区凤阳街道我的邻居遭遇了和你一样的事情通知阳性显示检测中然后下午来人拉走了
...,...,...,...,...,...,...,...,...,...,...
195,195,chn_dev_track_a_00196,0,0,0,0,0,0,chn,那些变化着的感染数据是对防控政策最好的评价其他都是瞎扯另外经济发展的好坏更是和感染数据直接相...
196,196,chn_dev_track_a_00197,0,0,0,0,0,0,chn,愿你眼中有美心中有爱遇见美好撞见幸福每一天都是春暖花开 green heart
197,197,chn_dev_track_a_00198,0,0,0,0,0,0,chn,在这撩人的五月初夏季节里不负时光不负己对于日复一日的每一天有人把它叫做日子也有人把它叫做生活...
198,198,chn_dev_track_a_00199,0,0,0,0,0,0,chn,哪个无良媒体先带头炒的流量就给人家赔偿还有这帮孙子是不是看见啥都想摸


In [166]:
x_dev, y_dev = dev_data['clean_message'].values.tolist(), dev_data[label_columns].values.tolist()

In [167]:
y_dev_df = pd.DataFrame(y_dev, columns=label_columns)

In [168]:
x_dev_embeddings = extract_embeddings(x_dev, embedding_model, batch_size=32)

Extracting embeddings: 100%|██████████| 7/7 [00:06<00:00,  1.11it/s]


In [169]:
y_dev_flat = np.array(y_dev)

In [170]:
# Predict labels for the test data
test_predictions = {}
for label in label_columns:
    print(f"Predicting for label: {label}")
    test_predictions[label] = svm_models[label].predict(x_dev_embeddings)

Predicting for label: anger
Predicting for label: disgust
Predicting for label: fear
Predicting for label: joy
Predicting for label: sadness
Predicting for label: surprise


In [171]:
predictions_df = pd.DataFrame(test_predictions)
dev_data.reset_index(inplace=True)

In [172]:
output_df = pd.concat([dev_data[['id','lang']], predictions_df], axis=1)

In [173]:
!rm -rf track*

In [174]:
# Create directory if it doesn't exist
output_dir = task
os.makedirs(output_dir, exist_ok=True)

# Iterate through the unique languages in the 'lang' column
for lang in output_df['lang'].unique():
    # Filter rows for the current language
    lang_df = output_df[output_df['lang'] == lang]
    lang_df.drop(columns=['lang'],inplace=True)

    # Define the output file path for the current language
    lang_csv_file = os.path.join(output_dir, f"pred_{lang}.csv")

    # Sort the DataFrame by the 'id' column
    lang_df.sort_values(by='id', inplace=True)

    # Save the filtered DataFrame to a CSV file
    lang_df.to_csv(lang_csv_file, index=False)

    # Print a confirmation message
    print(f"Predictions for language '{lang}' saved to {lang_csv_file}")

Predictions for language 'chn' saved to track_a/pred_chn.csv


In [175]:
!zip -r {task}.zip {task}

  adding: track_a/ (stored 0%)
  adding: track_a/pred_chn.csv (deflated 90%)
